In [ ]:
# default_exp core

# Core

> Main functionality.

The implementation of the main script is actually in the `build` function below. `main` is just a *wrapper* that parses command-line arguments.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import sys
import argparse
import pathlib
import importlib.util
import string
import collections
from types import ModuleType
from typing import Optional, Union

import numpy as np
import yaml

# to avoid tqdm's experimental warning
import warnings
warnings.filterwarnings("ignore", message='Using `tqdm.autonotebook.tqdm` in notebook mode')

import py2gift.util
import py2gift.question
import py2gift.input_file

import gift_wrapper.core
import gift_wrapper.question

# some classes in `gift_wrapper.question` are "patched" when this module is imported
import py2gift.markdown

## Command-line arguments

The function below just parses command-line arguments and pass them to the `build` function below.

In [ ]:
# export
def main():
    
    parser = argparse.ArgumentParser(description='Python to GIFT converter')

    parser.add_argument(
        'input_file', type=argparse.FileType('r'), default='global_settings.yaml', help='settings file', nargs='?')

    parser.add_argument('-c', '--code_directory', default='.', help='directory with the required source code')

    parser.add_argument(
        '-m', '--main_module', default='questions.py', help='file with the questions generators')

    parser.add_argument(
        '-l', '--local', default=False, action='store_true', help="don't try to copy the images to the server")

    command_line_arguments = parser.parse_args()
    
    code_directory = pathlib.Path(command_line_arguments.code_directory)
    main_module = pathlib.Path(command_line_arguments.main_module)

    sys.path.insert(0, code_directory.absolute().as_posix())
    spec = importlib.util.spec_from_file_location(main_module.stem, (code_directory / main_module).absolute())
    questions_generators = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(questions_generators)
    
    build(command_line_arguments.input_file.name, command_line_arguments.local, questions_generators)

## Auxiliary functions

### Obtaining the `__init__` parameters for a class from the corresponding settings

A function to obtain the initialization parameters of a given class given the corresponding settings.

* `cls_settings`: a `dict` with the all parameters for the class; it should include keys `statement`, `feedback` and, optionally, `time`

It returns another dictionary.

In [ ]:
# export
def init_parameters_from_settings(cls_settings: dict) -> dict:
    """
    Returns a dictionary with the initialization parameters for a question.
    
    **Parameters**
    
    - cls_settings: dict
        
        Settings for the class, which should include `statement`, `feedback` and, optionally, `time`.
        
    **Returns**
    
    - out: dict
        
        A dictionary with the *exact* parameters that must be passed when instantiating the class.
    """

    init_parameters = {
        'statement': py2gift.question.TemplatedLatexText(cls_settings['statement']),
        'feedback': py2gift.question.TemplatedLatexText(cls_settings['feedback'])
    }
    
    if 'time' in cls_settings:
        
        init_parameters['time'] = cls_settings['time']

    init_parameters.update(cls_settings.get('init parameters', {}))
    
    return init_parameters

The name for a *testing* file

In [ ]:
input_file = '_input.yaml'

In [ ]:
%%writefile {input_file}

output file: quiz.yaml
pictures base directory: tc/midterm3

categories:

  - name: Test category

    classes:

      - name: TestClass

        question base name: Test class
            
        init parameters:
            
            distribution: Gaussian

        statement: |
          Consider a random variable, $X$, with mean $ \mu = !mean $ and variance...
          

        feedback: |
          Clearly, $Y$ is...
          

        time: 15
        
        number of instances: 2

Overwriting _input.yaml


The *initialization parameters* for the first class in the first category can be obtained as

In [ ]:
input_data = py2gift.util.yaml_to_dict(input_file)    
init_parameters_from_settings(input_data['categories'][0]['classes'][0])

{'statement': un-filled template:
 Consider a random variable, $$X$$, with mean $$ \mu = $mean $$ and variance...,
 'feedback': Clearly, $Y$ is...,
 'time': 15,
 'distribution': 'Gaussian'}

If `init_parameters` is absent, then no parameters will be passed when instatiating the class.

## Processing of a bunch of questions

### Building a *GIFT* file from generators' settings

The workhorse of the library.

In [ ]:
# export
def build(
    settings: Union[str, dict], local_run: bool, questions_module: ModuleType,
    parameters_file: Union[str, dict] = 'parameters.yaml', no_checks: bool = False,
    overwrite_existing_latex_files: bool = True, embed_images: bool = False) -> None:
    """
    Generates a GIFT file.

    **Parameters**
    
    - settings: str or dict
        
        Settings for all the questions (generators).
    - local_run: bool
        
        If True, pictures will not be copied to a remote host.
    - questions_module: ModuleType
        
        A module or structure that holds the classes referenced in the settings.
    - parameters_file: str or dict
        
        File or dictionary with the parameters for "gift-wrapper".
    - no_checks: bool
        
        Whether or not LaTeX formulas should be checked.
    - overwrite_existing_latex_files: bool
        
        If True the auxiliar file for checks should be, if existing, overwritten without a warning.
    - embed_images: bool
        
        If True, images will be embedded in the questions (rather than linked).
    """
    
    # if settings is the name of a file...
    if type(settings) == str:

        with open(settings) as f:

            settings = yaml.load(f, Loader=yaml.FullLoader)
    
    else:
        
        assert type(settings) == dict

    output_file = settings['output file']

    category_questions = collections.defaultdict(list)

    for cat in settings['categories']:

        questions = []

        for c in cat['classes']:

            this_class_questions = []
            
            # either `parameters` or `number of instances` is present, but not both
            assert ('parameters' in c) ^ ('number of instances' in c), (
                'either "parameters" or "number of instances" must be specified')
            
            question_generator = getattr(questions_module, c['name'])(**init_parameters_from_settings(c))
            
            if 'parameters' in c:

                for p in c['parameters']:

                    this_class_questions.append(question_generator(**p))
            
            else:
                
                for _ in range(c['number of instances']):
                    
                    this_class_questions.append(question_generator())

            questions.extend(py2gift.util.add_name(this_class_questions, base_name=c['question base name']))
        
        category_questions[cat['name'] if type(cat['name']) != list else tuple(cat['name'])].extend(questions)

    # --------

    py2gift.util.write_multiple_categories(
        category_questions, settings['pictures base directory'], output_file=output_file)
    
    gift_wrapper.core.wrap(
        parameters_file, output_file, local_run=local_run, no_checks=no_checks,
        overwrite_existing_latex_files=overwrite_existing_latex_files, embed_images=embed_images)

This function expects, among other things, an *input file* (defined above), a *module* implementing the classes referenced in the latter, and a *parameters file*.

#### Module with questions

This can be used in the same way as module would.

In [ ]:
class FakeModule:
    
    class TestClass(py2gift.question.NumericalQuestionGenerator):
    
        def __init__(
            self, statement: py2gift.question.TemplatedLatexText, feedback: py2gift.question.TemplatedLatexText,
            distribution: str,
            time: Optional[int] = None,prng: np.random.RandomState = np.random.RandomState(42)) -> None:

            super().__init__(statement, feedback, time, prng)
            
            self.distribution = distribution
        
        def setup(self):
            
            mean = np.random.rand()

            self.statement.fill(mean=mean)

            self.solution = 42
            self.error = '10%'

#### Parameters file

In [ ]:
parameters_file = '_parameters.yaml'

In [ ]:
%%writefile {parameters_file}
images hosting:

  ssh:
    user: mvazquez

    # below, one should specify either a password for the user or  "public key" file but NOT both of them

    password:

    # "~" stands for the user's home directory (in Linux for one...)
    public_key: ~/.ssh/id_rsa_mymachine.pub

  copy:
    # machine into which files will be copied
    host: hidra1

    # the path that in the remote machine acts as root of the publicly visible directories hierarchy (hence it's not
    # visible from outside);  it *should* exist ("." stands for the working directory when you ssh into the machine)
    public filesystem root: ./public_html

  # public address from which the images will hang
  public URL: http://www.tsc.uc3m.es/~mvazquez/

latex:

  # auxiliary TeX file that will be created to check that formulas can be compiled
  auxiliary file: __latex_check.tex

Overwriting _parameters.yaml


In [ ]:
build(input_file, local_run=True, questions_module=FakeModule, parameters_file=parameters_file, no_checks=True)



file "quiz.gift.txt" created


The file actually created underneath by `build` (the name, *quiz.yaml*, was set above in the `input_file`)

In [ ]:
%pycat quiz.yaml

pictures base directory: tc/midterm3
categories:
  - name: Test category
    questions:
      - class: Numerical
        statement: "Consider a random variable, $X$, with mean $ \\mu = 0.597 $ and\
          \ variance...\n"
        feedback: "Clearly, $Y$ is...\n"
        time: '15'
        solution:
          value: 42
          error: 10%
        name: Test class I
      - class: Numerical
        statement: "Consider a random variable, $X$, with mean $ \\mu = 0.189 $ and\
          \ variance...\n"
        feedback: "Clearly, $Y$ is...\n"
        time: '15'
        solution:
          value: 42
          error: 10%
        name: Test class II


is ultimately processed by `gift_wrapper` to yield

In [ ]:
%pycat quiz.gift.txt

$CATEGORY: $course$/Test category

::Test class I::[html]Consider a random variable, \\(X\\), with mean \\( \\mu \= 0.597 \\) and variance...<br><br><br><i>Estimated time\: 15 minutes</i><br>{
#       =%100%42:4.2#
        ####Clearly, \\(Y\\) is...
}

::Test class II::[html]Consider a random variable, \\(X\\), with mean \\( \\mu \= 0.189 \\) and variance...<br><br><br><i>Estimated time\: 15 minutes</i><br>{
#       =%100%42:4.2#
        ####Clearly, \\(Y\\) is...
}


This is the file that is actually imported from *Moodle*, and *quiz.yaml* is mostly a *side effect*.

In [ ]:
!rm quiz.yaml quiz.gift.txt

One can also pass directly the `dict` with the settings.

In [ ]:
settings = py2gift.util.yaml_to_dict(input_file)

build(settings, local_run=True, questions_module=FakeModule, parameters_file=parameters_file, no_checks=True)



file "quiz.gift.txt" created


In [ ]:
%pycat quiz.yaml

pictures base directory: tc/midterm3
categories:
  - name: Test category
    questions:
      - class: Numerical
        statement: "Consider a random variable, $X$, with mean $ \\mu = 0.0323 $ and\
          \ variance...\n"
        feedback: "Clearly, $Y$ is...\n"
        time: '15'
        solution:
          value: 42
          error: 10%
        name: Test class I
      - class: Numerical
        statement: "Consider a random variable, $X$, with mean $ \\mu = 0.476 $ and\
          \ variance...\n"
        feedback: "Clearly, $Y$ is...\n"
        time: '15'
        solution:
          value: 42
          error: 10%
        name: Test class II


## Processing a single question

This is mostly intended for previewing a question in a [jupyter](https://jupyter.org/) notebook.

### Building a single question

This function returns settings that allow building a question using *gift-wrapper*.

In [ ]:
# export
def build_question(
    question_generator: py2gift.question.QuestionGenerator, category_name: str, settings: dict, n_question: int=0
) -> dict:
    """
    Returns the settings for building a question using "gift-wrapper".
    
    Parameters
    ----------
    question_generator: class
        The question generator that will generate the appropirate settings.
    category_name: str
        The name of category the class belongs to.
    settings: dict
        User settings.
    n_question: int
        The number of instance to be returned.
        
    Returns
    -------
    out: dict
        A dictionary with the settings that allow building the question using "gift-wrapper".
        
    """
    
    class_name = question_generator.__name__
    
    class_settings = py2gift.input_file.extract_class_settings(category_name, class_name, settings)
    
    # an instance
    question_generator = question_generator(**init_parameters_from_settings(class_settings))

    assert ('parameters' in class_settings) ^ ('number of instances' in class_settings), (
        'either "parameters" or "number of instances" must be specified')

    if 'parameters' in class_settings:
        # notice that `parameters` yields a list
        return question_generator(**class_settings['parameters'][n_question])
    else:
        return question_generator()

For testing, we need some settings. It is more natural to read and write them in a file.

In [ ]:
settings_file = '_settings.yaml'

In [ ]:
%%writefile {settings_file}

output file: quiz.yaml
pictures base directory: pics

categories:

  - name: Category 1

    classes:

      - name: A

        question base name: A numerical question

        statement: What is...

        feedback: Well...

        parameters:

          - a: 5
            b: 3

      - name: A

        question base name: Another numerical question

        statement: What is...

        feedback: Well...
        
        number of instances: 2

  # -----------------------------

  - name: Category 2

    classes:

      - name: B

        question base name: A multiple-choice question

        init parameters:

          nodes: ['S1', 'S2']

        statement: |
          Consider...

        feedback: |
          We must...

        parameters: &dijkstra_parameters

          - n: 0

          - n: 1
            
  - name: Category 3

    classes:

      - name: C

        question base name: Another numerical question

        statement: What is...

        feedback: Well...
        
        number of instances: 2

Overwriting _settings.yaml


Next, we need to define the classes referenced above (`A`, `B` and `C`)

In [ ]:
class A(py2gift.question.NumericalQuestionGenerator):
    
    def setup(self, a: int, b:int):
        
        self.solution = 42
        self.error =  '10%'

class B(py2gift.question.MultipleChoiceQuestionGenerator):
    
    def __init__(
        self, statement: py2gift.question.TemplatedLatexText, feedback: py2gift.question.TemplatedLatexText,
        nodes: list,
        time: Optional[int] = None, prng: np.random.RandomState = np.random.RandomState(42)) -> None:
        
        super().__init__(statement, feedback, time, prng)
        
        self.nodes = nodes
    
    
    def setup(self, n:int):
        
        self.wrong_answers = [
            ['yes', 50],
            ['no', -50],
            ['nope', -50],
            ['yessss!!!', 50]
        ]

class C(py2gift.question.NumericalQuestionGenerator):
    
    def setup(self):
        
        self.solution = 52
        self.error =  '10%'

In [ ]:
build_question(A, category_name='Category 1', settings=py2gift.util.yaml_to_dict(settings_file))

{'class': 'Numerical',
 'statement': 'What is...',
 'feedback': 'Well...',
 'solution': {'value': 42, 'error': '10%'}}

In [ ]:
build_question(C, category_name='Category 3', settings=py2gift.util.yaml_to_dict(settings_file))

{'class': 'Numerical',
 'statement': 'What is...',
 'feedback': 'Well...',
 'solution': {'value': 52, 'error': '10%'}}

In [ ]:
# !rm {settings_file}

## Previewing

The settings of a question (in the form of a dictionary),

In [ ]:
question_settings = {
    'class': 'Numerical',
    'statement': 'What is the value of $\\pi$?',
    'solution': {'value': 3.14, 'error': '50%'},
    'feedback': 'well, $\\pi$'
}

, can be turned into a markdown string by `question.settings_to_markdown`

In [ ]:
py2gift.util.render_latex(py2gift.markdown.settings_to_markdown(question_settings))


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

What is the value of $\Large \pi$?


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



well, $\Large \pi$

<span style="font-family:Papyrus; font-size:2em;">Solution</span>

 3.14 (error: 1.57)


In [ ]:
# export
def generator_to_markdown(
    settings: Union[str, pathlib.Path, dict], category: str, cls: py2gift.question):
    """
    Returns markdown text from a generator.


    Parameters
    ----------
    settings: str, Pathlib, dict
        The settings file or corresponding dictionary.
    category: str
        The category of the question.
    cls: py2gift.question.QuestionGenerator
        The class implementing the generator.

    Returns
    -------
    out: str
        Markdown text.

    """
    
    # if settings is the name of a file...
    if type(settings) == str:
        
        settings = py2gift.util.yaml_to_dict(settings)
    
    else:
        
        assert type(settings) == dict

    question_settings = build_question(cls, category, settings)

    return py2gift.markdown.settings_to_markdown(question_settings)

In [ ]:
py2gift.util.render_latex(generator_to_markdown(settings_file, category='Category 1', cls=A))


<span style="font-family:Papyrus; font-size:2em;">Statement</span>

What is...


<span style="font-family:Papyrus; font-size:2em;">Feedback</span>



Well...

<span style="font-family:Papyrus; font-size:2em;">Solution</span>

 42 (error: 4.2)


## Preprocessing tex files

Function to turn a $\TeX$ file into an svg. It returns a *markdown* string that allows to visualize it in a cell.

In [ ]:
# export
def latex_to_markdown(input_file: Union[str, pathlib.Path], delete_input_file_afterwards: bool = False) -> str:
    """
    Returns markdown text that shows the result of compiling a TeX file.


    Parameters
    ----------
    input_file: str, Pathlib
        The TeX file.
    delete_input_file_afterwards: bool
        If True the TeX file is deleted after conversion to svg.

    Returns
    -------
    out: str
        Markdown text.

    """
    
    output_file = gift_wrapper.image.pdf_to_svg(gift_wrapper.image.tex_to_pdf(input_file))
    
    suffixes = ['.aux', '.log', '.pdf']
    
    if delete_input_file_afterwards:
        
        suffixes.append('.tex')
    
    for suffix in suffixes:
        
        file_to_delete = output_file.with_suffix(suffix)
        
        if file_to_delete.exists():
        
            file_to_delete.unlink()
    
    return r'![](' + output_file.as_posix() + ')'

A sample $\TeX$ file,

In [ ]:
tex_file = '_sample.tex'

, is written

In [ ]:
%%writefile {tex_file}
\documentclass[]{standalone}

\usepackage[utf8]{inputenc}

\usepackage{tikz}


\begin{document}

\begin{tikzpicture}
    \node[draw, text width=60, align=center,font=\small,minimum height=80] {The Ingenious Gentleman Don Quixote of La Mancha};
\end{tikzpicture}

\end{document}

Overwriting _sample.tex


However, you can only run the cell below if you have a $\LaTeX$ installation with *TikZ*

In [ ]:
#py2gift.util.render_latex(latex_to_markdown(tex_file, delete_input_file_afterwards=True))

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('00_core.ipynb')

Converted 00_core.ipynb.
